In [11]:
import requests
import re
import hashlib
import json
import time
import hashlib
import random
from pyquery import PyQuery as pq

In [42]:
data=[]

In [46]:
url = 'https://www.instagram.com/explore/locations/237094598/'
uri = 'https://www.instagram.com/graphql/query/?query_hash=1b84447a4d8b6d6d0426fefb34514485&variables=%7B%22id%22%3A%22{user_id}%22%2C%22first%22%3A12%2C%22after%22%3A%22{cursor}%22%7D'

In [14]:
headers = {
'Connection':'keep-alive',
'Host':'www.instagram.com',
'Referer':'https://www.instagram.com/',
'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/68.0.3440.106 Safari/537.36',
'X-Requested-With':'XMLHttpRequest'
}

In [15]:
def hashStr(strInfo):
    h = hashlib.md5()
    h.update(strInfo.encode("utf-8"))
    return h.hexdigest()

In [16]:
def get_urls(url):
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            return response.text
        else:    
            print('请求错误状态码：', response.status_code)        
    except Exception as e:
        print(e)
        return None

In [17]:
def get_json(url,headers):
    try:
        response = requests.get(url, headers=headers, timeout=10)
        if response.status_code == 200:
            return response.json()
        else:
            print('请求网页json错误, 错误状态码：', response.status_code)
    except Exception as e:
        print(e)
        time.sleep(60 + float(random.randint(1, 4000))/100)
        return get_json(url)

In [37]:
def crawl(html):
    j=0
    profile = {}
    doc = pq(html)
    items = doc('script[type="text/javascript"]').items()
    
    user_id = re.findall('"locationPage_([0-9]+)"', html, re.S)[0]
    #print(user_id )
    GIS_rhx_gis = re.findall('"rhx_gis":"([0-9a-z]+)"', html, re.S)[0]
    
    for item in items:
        if item.text().strip().startswith('window._sharedData'):
            js_data = json.loads(item.text()[21:-1], encoding='utf-8')
            edges = js_data["entry_data"]["LocationsPage"][0]["graphql"]["location"]["edge_location_to_media"]["edges"]
            page_info = js_data["entry_data"]["LocationsPage"][0]["graphql"]["location"]["edge_location_to_media"]['page_info']
            cursor = page_info['end_cursor']
            flag = page_info['has_next_page']
            
            for edge in edges:
                try:
                    profile['id'] = edge['node']['owner']['id']
                except:
                    profile['id']
                    continue
                    
                try:
                    profile['text'] = edge['node']['edge_media_to_caption']['edges'][0]['node']['text']
                except:
                    profile['text'] = "None"
                    continue
                    
                try:
                    profile['picLink'] = edge['node']['display_url']
                except:
                    profile['picLink'] = "None"
                    continue
                    
                try:
                    profile['count'] = edge['node']['edge_liked_by']['count']
                except:
                    profile['count'] = "None"
                    continue
                    
                try:
                    profile['comment'] = edge['node']['edge_media_to_comment']['count']
                except:
                    profile['comment'] = "None"
                    continue
                    
                data.append(profile)
                profile = {}
                
    
    while flag:
        url_1 = uri.format(user_id=user_id, cursor=cursor)
        queryVariables = '{"id":"' + user_id + '","first":12,"after":"' +cursor+ '"}'
        headers['X-Instagram-GIS'] = hashStr(GIS_rhx_gis + ":" + queryVariables)
                
        js_data = get_json(url_1,headers)
        infos = js_data['data']["location"]["edge_location_to_media"]["edges"]
        cursor = js_data['data']["location"]["edge_location_to_media"]['page_info']['end_cursor']
        flag = js_data['data']["location"]["edge_location_to_media"]['page_info']['has_next_page']
            
        for info in infos:
            try:
                profile['id'] = info ['node']['owner']['id']
            except:
                profile['id'] = "None"
                continue
                
            try:
                profile['text'] = info ['node']['edge_media_to_caption']['edges'][0]['node']['text']
            except:
                profile['text'] = "None"
                continue
                
            try:
                profile['picLink'] = info ['node']['display_url']
            except:
                profile['picLink'] = "None"
                continue
                
            try:
                profile['count'] = info ['node']['edge_liked_by']['count']
            except:
                profile['count'] = "None"
                continue
                
            try:
                profile['comment'] = info ['node']['edge_media_to_comment']['count']
            except:
                profile['comment'] = "None"
                continue
                
            data.append(profile)
            profile = {}
        print(j)
        j=j+1
        time.sleep(3)
    
            
    return profile

In [51]:
html = get_urls(url)
crawl(html)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78


{}

In [48]:
j=0
for i in data:
    print(j)
    print('id:'+i['id']+'\n')
    print('picturelink:'+'\n'+i['picLink']+'\n')
    print('text:'+'\n'+i['text']+'\n')
    print('like:')
    print(i['count'])
    print('comment:')
    print(i['comment'])
    print("===========================================================================================")
    print("\n")
    j=j+1
    
        

0
id:2001312927

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/c19580dd2f6ae72091039cc239f3908c/5CC56874/t51.2885-15/e35/50813759_611391262646228_5757033012021485223_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
台南孔廟魅力商圈⛩

#台南#台南景點#府城#孔廟#孔廟商圈

like:
3
comment:
0


1
id:4177443350

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/79ded652abe59b4f93cb39fbdaa2a845/5CCD186B/t51.2885-15/e35/49745792_2241648116087905_2480138427775270000_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
獨自一人
#景色#絕景

like:
10
comment:
1


2
id:3134126169

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/77ffdc2bbcf57530a9b8f60229580f12/5CE0BBBB/t51.2885-15/e35/50515173_235182004054383_8854516495608420438_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
農曆過年快到了
歡迎來把哩賀帶回家

#豬年 #貼紙 #免費索取
#吉來 #吉來小早餐 
#台南 #府中街 #孔廟
#吐司 #漢堡 #蛋餅
#仙女紅茶 #仙奶茶
#真檸檬紅茶
#黑咖啡 #拿鐵

like:
21
comment:
2


3
id:4369048177

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/546593373440be534b09c2710508d15c/5

3


237
id:563789838

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/d22e744fa5a42d8c0a2f7f7740c09e4f/5CDE2D0F/t51.2885-15/e35/37185053_250474448900568_6728355904598573056_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
原來這裡這麼好拍

like:
112
comment:
5


238
id:4238215837

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/20e554ab565b39e697fa143cf0649c5c/5CBA42C1/t51.2885-15/e35/36993137_241020246719630_7544951899078787072_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
🔍🔍Fb:amuse美式時尚
💻Line:@mvd7413y（需加@）
🏠🏠台南市青年路78-1號

#台南#台灣代購 #台北 #台中#高雄#屏東#taipei #taiwan🇹🇼 #tainan #taipei #歡迎詢問 #青年路78 #單品 #美貨 #現貨 #海外旅行大好き#桃園#嘉義#台東#花蓮#宜蘭#鹿港#南投#臺南#臺南美食#2eat2gether #like4like #tainanlife

like:
15
comment:
0


239
id:2915121338

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/32a6384174fdaae68781ea0525606e3f/5CC1E802/t51.2885-15/e35/36940214_405530516519550_4357485802033250304_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
炒泡麵好吃💓
#保哥黑輪

like:
23
comment:
0


240
id:423

2


437
id:2377222140

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/3944aad28138acd44fed2c5de307694d/5CDD1529/t51.2885-15/e35/27891014_1079796568826887_53800422502039552_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
府中街。

like:
45
comment:
2


438
id:5419185028

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/7a12a6ab20fe2a48d89be441f5449574/5D00202D/t51.2885-15/e35/27891070_1605742759507064_3309669014812229632_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
Edible Totoro-buns.  Not edible piggy bank (it does eat your coins not larger than NT$10). 吊飾品："好事馬上花生(發生)"
#童話閣 #假日休閒 #TainanToDo

like:
5
comment:
0


439
id:1431500179

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/2488c53e614a2d42ea94cc3e85f3c519/5CFEA0D5/t51.2885-15/e35/27582260_224197604820750_188144418245574656_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
#travelpic #traveller #travel #vacation #vacationmode #girlsday #girlgang #girl #girls #tourism #scenery #sceneryporn #friends

comment:
0


637
id:178740720

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/5f602688ba324d9b3095e5b926cb4165/5CCDFCE1/t51.2885-15/e35/23161419_906356579527526_2419288866620964864_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
Tainan foi dominada por diversos povos e impérios. Iniciando-se pelo controle das Cia das Índias Orientais Holandesas que, após inúmeros conflitos, os Kuchingas (aborígenes) retomaram seu controle. Após o imperador chinês em seu reinado Qing tomou controle até que o exército imperial invadisse essa porção de Taiwan durando até a rendição nipônica ao final da WWII #taiwan #heartofasia #tainantrip #fortprovintia #fortzeelandia #hayashidepartmentstore #asiatrip

like:
46
comment:
0


638
id:199337269

picturelink:
https://instagram.fkhh1-2.fna.fbcdn.net/vp/4670e60237e2cdd395f061364d3da526/5CFCC8A1/t51.2885-15/e35/23164741_720867488107368_288219050322952192_n.jpg?_nc_ht=instagram.fkhh1-2.fna.fbcdn.net

text:
特技表演，雙腿健壯
臺南阿啾

like:
10
comment:
1


639
id

In [52]:
def file(data):
    with open("instaCrawler_台南林百貨",mode="w",encoding="utf-8")as file:
        json_string=json.dumps(data)
        file.write(json_string)

In [53]:
file(data)